In [8]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from statsmodels import robust

In [9]:
SIZE = 1000
DIMENSION = 5000

In [10]:
normal_distrib = np.random.normal(0,1, size=(DIMENSION,SIZE))
coshi_distrib = np.random.standard_cauchy(size=(DIMENSION,SIZE))
mix_distrib =  0.9 * np.random.normal(0, 1, size=(DIMENSION,SIZE)) + \
                  0.1 * np.random.standard_cauchy(size=(DIMENSION,SIZE))

In [11]:
#plt.hist(normal_distrib[0])

In [12]:
len(normal_distrib[0])

1000

In [13]:
def my_huber(data):
    x = np.mean(data)
    if abs(x) < 1.44:
        return x
    else:
        return (-1.44,1,44)[x>1.44]
    
def two_stage_estimator_fun(data):
    q1 = np.percentile(data, 25)
    q3 = np.percentile(data, 75)

    # Вычисление IQR
    iqr = q3 - q1

    # Определение границ для выбросов
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    # Отбрасывание выбросов
    filtered_data = data[(data >= lower_bound) & (data <= upper_bound)]

    return np.mean(filtered_data)


In [14]:

means = {'norm':[], 'coshi':[], 'mix':[]}
medians = {'norm':[], 'coshi':[], 'mix':[]}
huber_estimates = {'norm':[], 'coshi':[], 'mix':[]}
two_stage_estimates = {'norm':[], 'coshi':[], 'mix':[]}
for i in range (DIMENSION):
    means['norm'].append(np.mean(normal_distrib[i]))
    means['coshi'].append(np.mean(coshi_distrib[i]))
    means['mix'].append(np.mean(mix_distrib[i]))
    
    medians['norm'].append(np.median(normal_distrib[i]))
    medians['coshi'].append(np.median(coshi_distrib[i]))
    medians['mix'].append(np.median(mix_distrib[i]))
    
    huber_estimates['norm'].append(my_huber(normal_distrib[i]))
    huber_estimates['coshi'].append(my_huber(coshi_distrib[i]))
    huber_estimates['mix'].append(my_huber(mix_distrib[i]))
    
    two_stage_estimates['norm'].append(two_stage_estimator_fun(normal_distrib[i]))
    two_stage_estimates['coshi'].append(two_stage_estimator_fun(coshi_distrib[i]))
    two_stage_estimates['mix'].append(two_stage_estimator_fun(mix_distrib[i]))
    

/tmp/ipykernel_4317/2263172887.py:6: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return (-1.44,1,44)[x>1.44]


In [15]:
#Нормальное распределение 
def res_print(types):
    names = {types[0]: "Нормальное:", types[1]: "Коши:", types[2]: "Смесь:"}
    for type_ in types:
        print(f'\n{names[type_]}')
        print("\tСреднее:")
        print("\t\t\tСреднее:\t", f"{np.mean(means[type_]):.6f}", "\n", "\t\t\tДисперсия:\t", f"{np.var(means[type_]):.6f}")

        print("\tМедиана:")
        print("\t\t\tСреднее:\t", f"{np.mean(medians[type_]):.6f}","\n", "\t\t\tДисперсия:\t", f"{np.var(medians[type_]):.6f}")

        print("\tХубер:")
        print("\t\t\tСреднее:\t", f"{np.mean(huber_estimates[type_]):.6f}","\n" , "\t\t\tДисперсия:\t", f"{np.var(huber_estimates[type_]):.6f}")

        print("\tДвухэтапная оценка:")
        print("\t\t\tСреднее:\t", f"{np.mean(two_stage_estimates[type_]):.6f}","\n", "\t\t\tДисперсия:\t", f"{np.var(two_stage_estimates[type_]):.6f}")
res_print(['norm', 'coshi', 'mix'])


Нормальное:
	Среднее:
			Среднее:	 -0.000500 
 			Дисперсия:	 0.001019
	Медиана:
			Среднее:	 -0.000763 
 			Дисперсия:	 0.001633
	Хубер:
			Среднее:	 -0.000500 
 			Дисперсия:	 0.001019
	Двухэтапная оценка:
			Среднее:	 -0.000437 
 			Дисперсия:	 0.001094

Коши:
	Среднее:
			Среднее:	 -0.361394 
 			Дисперсия:	 845.544835
	Медиана:
			Среднее:	 0.000527 
 			Дисперсия:	 0.002466
	Хубер:
			Среднее:	 -0.080823 
 			Дисперсия:	 0.880205
	Двухэтапная оценка:
			Среднее:	 0.000400 
 			Дисперсия:	 0.003397

Смесь:
	Среднее:
			Среднее:	 -0.090858 
 			Дисперсия:	 17.706086
	Медиана:
			Среднее:	 -0.001097 
 			Дисперсия:	 0.001492
	Хубер:
			Среднее:	 -0.007892 
 			Дисперсия:	 0.149551
	Двухэтапная оценка:
			Среднее:	 -0.000271 
 			Дисперсия:	 0.001055


In [16]:
def get_graph(dict_,name):
    name_arr = ['norm', 'coshi', 'mix']
    fig, ax = plt.subplots(nrows=3, ncols=1,figsize=(15,15))

    for i,type_ in enumerate(dict_.keys()):
        ax[i].set_title(name_arr[i])
        ax[i].plot(dict_[type_])
    fig.suptitle(name,fontsize=20)
    plt.show()

In [20]:
def get_hist(dict_,name):
    name_arr = ['norm', 'coshi', 'mix']
    fig, ax = plt.subplots(nrows=3, ncols=1,figsize=(15,15))

    for i,type_ in enumerate(dict_.keys()):
        ax[i].set_title(name_arr[i])
        ax[i].hist(dict_[type_],bins=20)
    fig.suptitle(name,fontsize=20)
    plt.show()